In [13]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import time


class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4* config.n_embd)
        self.gelu = nn.GELU(approximate = "tanh") # remove approximate, because today exact works also
        self.c_proj = nn.Linear(4*config.n_embd, config.n_embd)
    
    def forward(self,x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
    
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

class GPT(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd), # context fenster
            h = nn.ModuleList(Block(config) for _ in range(config.n_layer)),
            ln_f = nn.LayerNorm(config.n_embd),
        )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias = False)
    
    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss
        # return logits
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


import tiktoken


class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens from disk and store them in memory
        with open('goethe.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y



#train_loader = DataLoaderLite(B=4, T=32) # B = 16, T = 1024
train_loader = DataLoaderLite(B=8, T=512) # B = 16, T = 1024

device = "mps"

num_rt_seqs = 1
max_len  = 100

#model = GPT.from_pretrained("gpt2")
model = GPT(GPTConfig)
model.to(device)
#model = torch.compile(model)

enc = tiktoken.get_encoding("gpt2")



optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4, betas = (0.9, 0.95), eps= 10e-8)
for i in range(500):
    t0=time.time()
    optimizer.zero_grad()
    x, y = train_loader.next_batch()
    x, y= x.to(device), y.to(device)
    with torch.autocast(device_type=device, dtype = torch.bfloat16):
        logits,loss = model(x,y)
    loss.backward()
    optimizer.step()
    dt = time.time() - t0
    tokens_processed = train_loader.B * train_loader.T
    tokens_per_sec = tokens_processed / dt
    print(f"step {i}, loss: {loss.item()}, dt: {dt*1000:.2f} ms, tokens/sec: {tokens_per_sec:.2f}")


loaded 66404 tokens
1 epoch = 16 batches
step 0, loss: 10.951690673828125, dt: 502.76 ms, tokens/sec: 8146.98
step 1, loss: 8.68541145324707, dt: 468.52 ms, tokens/sec: 8742.45
step 2, loss: 8.050169944763184, dt: 466.82 ms, tokens/sec: 8774.32
step 3, loss: 7.666171073913574, dt: 468.19 ms, tokens/sec: 8748.59
step 4, loss: 7.268510341644287, dt: 463.17 ms, tokens/sec: 8843.42
step 5, loss: 7.099575042724609, dt: 468.24 ms, tokens/sec: 8747.59
step 6, loss: 7.095156669616699, dt: 465.50 ms, tokens/sec: 8799.12
step 7, loss: 6.991485118865967, dt: 467.28 ms, tokens/sec: 8765.62
step 8, loss: 6.6679158210754395, dt: 466.27 ms, tokens/sec: 8784.53
step 9, loss: 6.435262680053711, dt: 467.97 ms, tokens/sec: 8752.70
step 10, loss: 6.336226463317871, dt: 467.33 ms, tokens/sec: 8764.61
step 11, loss: 5.899265289306641, dt: 468.63 ms, tokens/sec: 8740.33
step 12, loss: 5.7912092208862305, dt: 465.39 ms, tokens/sec: 8801.17
step 13, loss: 6.009066104888916, dt: 464.95 ms, tokens/sec: 8809.50
s

In [14]:
model.eval()
model.to(device)
tokens = enc.encode("FAUST")
#tokens = enc.encode("Hi, who are you?")
tokens = torch.tensor(tokens, dtype = torch.long)
tokens = tokens.unsqueeze(0).repeat(num_rt_seqs, 1)
x = tokens.to(device)


#torch.manual_seed(42)

while x.size(1) < max_len:
    with torch.no_grad():
        logits,loss = model(x)
        logits = logits[:,-1,:]
        probs = F.softmax(logits, dim = -1)
        top_probs, top_ind = torch.topk(probs, 50, dim = -1)
        ix = torch.multinomial(top_probs,1)
        xcol = torch.gather(top_ind, -1, ix)
        x = torch.cat((x,xcol), dim = 1)
#
for i in range(num_rt_seqs):
    tokens = x[i, :max_len].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> FAUST’m near!
Would carry
They penetrate still we’
First wife to Adam.
Pay attention to her lovely hair,														
The presence,
Dancing.)
The deep into partnershipition –
Tutti 
At your finally. Oh,



Then take flight soon stream,														
The nature of
